In [ ]:
import sys
sys.path.append('./Modeling')
sys.path.append('./Data')

In [ ]:
# !pip install pyspark

In [ ]:
from regression import get_parquet_data, log_transformation, encoding_data, split_data, randomforest_grid_search, save_model, load_model, get_weather_data

In [ ]:
# 전처리 완료 데이터 불러오기
file_path = './Data/csv'

ride_df, alight_df = get_parquet_data(file_path)

In [ ]:
# 데이터프레임 형태 확인
ride_df.info()
alight_df.info()

# 왜도, 첨도 확인

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10, 10))
sns.distplot(ride_df['RIDE'])

plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.distplot(alight_df['ALIGHT'])

plt.show()

# 로그 변환, 스케일링, 인코딩

In [ ]:
# 로그 변환
ride_df = log_transformation(ride_df, RIDE)
alight_df = log_transformation(alight_df, ALIGHT)

In [ ]:
print("Skewness: {:.3f}".format(ride_df['RIDE_log'].skew()))
print("Kurtosis: {:.3f}".format(ride_df['RIDE_log'].kurt()))

print("Skewness: {:.3f}".format(alight_df['ALIGHT_log'].skew()))
print("Kurtosis: {:.3f}".format(alight_df['ALIGHT_log'].kurt()))

In [ ]:
# 스케일러, 인코딩 적용
ride_df_encoding = encoding_data(ride_df)
alight_df_encoding = encoding_data(alight_df)

# 훈련 데이터, 학습 데이터 분리

In [ ]:
ride_X_train, ride_y_train, ride_X_test, ride_y_test = split_data(ride_df_encoding, 'RIDE_log')
alight_X_train, alight_y_train, alight_X_test, alight_y_test = split_data(alight_df_encoding, 'alight_log')

# 최적 파라미터 구하기

In [ ]:
rf_best_model_ride = randomforest_grid_search(ride_X_train, ride_y_train)
rf_best_model_alight = randomforest_grid_search(alight_X_train, alight_y_train)

# 모델 저장

In [ ]:
save_model(rf_best_model_ride)
save_model(rf_best_model_alight)

# 승객 수 예측

In [ ]:
# 모델 불러오기
ride_loaded_model = load_model(rf_best_model_ride)
alight_loaded_model = load_model(rf_best_model_alight)

In [ ]:
# 기후 데이터 불러오기
pred_weather_df = get_weather_data(path, weather.csv)

# 기후 데이터 스케일러, 인코딩 적용
pred_weather_df = encoding_data(pred_weather_df)

# 기후 데이터를 바탕으로 승객 수 예측

In [ ]:
ride_predictions = ride_loaded_model.predict(pred_weather_df)
alight_predictions = alight_loaded_model.predict(pred_weather_df)

In [ ]:
# 로그 역변환
ride_reversed_values = np.exp(ride_predictions)
alight_reversed_values = np.exp(alight_predictions)

print("역변환된 값 : ", ride_reversed_values)
print("역변환된 값 : ", alight_reversed_values)